In [4]:
!pip install dill
!pip install nltk
!pip install cython
!pip install benepar[gpu]
!pip install tensorflow==1.14
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install flair

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.3.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.14.0
    Uninstall

In [9]:
# Load Libraries
from globalfn.annotations import all_annotations, annotation
from globalfn.alignments import all_alignments, aligned_with

import dill
import collections
import pathlib

import nltk
nltk.download('punkt')
nltk.download('framenet_v17')

import benepar
benepar.download('benepar_en2')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/framenet_v17.zip.
[nltk_data] Downloading package benepar_en2 to
[nltk_data]     /home/jovyan/nltk_data...


True

In [ ]:
def eval_res(gold, pred, print_result = True):
    """
    Calculate the following metrics:
    # Hamming Loss: fraction of FE labels that are incorrectly predicted (including wrong label)
    # Hard Hamming Loss: fraction of FE labels that are incorrectly predicted because of FP and FN
    # Soft Hamming Loss: fraction of FE labels that are incorrectly predicted because of wrong labels at the correct position
    # Exact Match Ratio: percentage of samples that have all their labels classified correctlys
    """
    hamming_wrong_labels = hamming_hard_wrong_labels = hamming_soft_wrong_labels = hamming_total_labels = 0
    dist_inexact = list()
    exact_match = exact_match_total = 0

    for tgt_sent_id, transferred_res in pred.items():
        gold_sent_res = gold[tgt_sent_id]
        for frameName in transferred_res.keys():
            if frameName in gold_sent_res:
                gold_sent_fe_res = gold_sent_res[frameName]
                pred_sent_fe_res = transferred_res[frameName]
                assert len(gold_sent_fe_res) == len(pred_sent_fe_res)

                # compute hamming loss
                for i in range(len(gold_sent_fe_res)):
                    hamming_total_labels += 1
                    hamming_wrong_labels += int(gold_sent_fe_res[i] != pred_sent_fe_res[i])

                    if gold_sent_fe_res[i] != pred_sent_fe_res[i] and (gold_sent_fe_res[i] == '-' or pred_sent_fe_res[i] == '-'):
                        hamming_hard_wrong_labels += 1
                    
                    if gold_sent_fe_res[i] != pred_sent_fe_res[i] and gold_sent_fe_res[i] != '-' and pred_sent_fe_res[i] != '-':
                        hamming_soft_wrong_labels += 1

                # compute exact match ratio
                exact_match_total += 1
                exact_match += int(gold_sent_fe_res == pred_sent_fe_res)

                dist_inexact.append((len(gold_sent_fe_res) - gold_sent_fe_res.count('-')) - (len(pred_sent_fe_res) - pred_sent_fe_res.count('-')))
                

    hamming_loss = hamming_wrong_labels/hamming_total_labels
    hard_hamming_loss = hamming_hard_wrong_labels / hamming_total_labels
    soft_hamming_loss = hamming_soft_wrong_labels / hamming_total_labels
    exact_match_ratio = exact_match/exact_match_total
    print(f"Total Hamming Loss: {hamming_loss:3f}")
    print(f"Hard Hamming Loss: {hard_hamming_loss:3f}")
    print(f"Soft Hamming Loss: {soft_hamming_loss:3f}")
    print(f"Exact Match Ratio: {exact_match_ratio:3f}")

In [ ]:
def compare_projection(gold, proj1, proj2):
    """
    Used for comparing the differences of various projection schemes when proj1 matches gold.
    @gold: gold frame element result (loaded from pickled file)
    @proj1: any projection result (loaded from pickled file)
    @proj2: any projection result (loaded from pickled file)
    """
    exact_match_total = exact_match = 0
    for tgt_sent_id, projection in proj1.items():
        gold_sent_res = gold[tgt_sent_id]
        for frameName in projection.keys():
            if frameName in gold_sent_res:
                gold_sent_fe = gold_sent_res[frameName]
                pred_sent_fe = projection[frameName]
                assert len(gold_sent_fe) == len(pred_sent_fe)

                # compute exact match ratio
                exact_match_total += 1
                exact_match += int(gold_sent_fe == pred_sent_fe)
                
                if gold_sent_fe == pred_sent_fe and proj2[tgt_sent_id][frameName] != gold_sent_fe:
                    print(tgt_sent_id, frameName, pred_sent_fe, proj2[tgt_sent_id][frameName])

### EN-DE Projections

In [ ]:
# gold frame elements label for DE sentences
gold_en2de = dill.load(open("results/gold_en2de_feproject.dill", 'rb'))

# constituent-to-constituent projection
c2c_en2de = dill.load(open("results/c2c_en2de_feproject.dill", 'rb')) 

# constituent-to-word projection
c2w_en2de = dill.load(open("results/c2w_en2de_feproject.dill", 'rb'))

# word-to-word projection
w2w_en2de = dill.load(open("results/w2w_en2de_feproject.dill", 'rb'))

In [ ]:
print("Word-to-Word Projection:")
eval_res(gold_en2de, w2w_en2de)
print("Constituent-to-Word Projection:")
eval_res(gold_en2de, c2w_en2de)
print("Constituent-to-Constituent projection:")
eval_res(gold_en2de, c2c_en2de)

Word-to-Word Projection:
Total Hamming Loss: 0.176975
Hard Hamming Loss: 0.162528
Soft Hamming Loss: 0.014447
Exact Match Ratio: 0.200000
Constituent-to-Word Projection:
Total Hamming Loss: 0.168397
Hard Hamming Loss: 0.161174
Soft Hamming Loss: 0.007223
Exact Match Ratio: 0.286957
Constituent-to-Constituent projection:
Total Hamming Loss: 0.193228
Hard Hamming Loss: 0.174266
Soft Hamming Loss: 0.018962
Exact Match Ratio: 0.217391


In [ ]:
compare_projection(gold_en2de, c2w_en2de, c2c_en2de)

1281 People ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Context_of_acquaintance', '-']
1290 Frequency ['-', '-', '-', '-', '-', '-', '-', '-'] ['-', 'Event', '-', 'Event', 'Event', '-', '-', '-']
1291 Typicality ['State_of_affairs', '-', '-', '-'] ['State_of_affairs', '-', 'fe_Experiencer', '-']
1295 Architectural_part ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Part', '-']
1295 Education_teaching ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', '-', 'Student', '-', '-', '-', '-', '-', '-', '-', '-']
1296 People ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-',

### EN-PT

In [ ]:
# gold frame elements label for PT sentences
gold_en2pt = dill.load(open("results/gold_en2pt_feproject.dill", 'rb'))

# constituent-to-constituent projection
c2c_en2pt = dill.load(open("results/c2c_en2pt_feproject.dill", 'rb'))

# constituent-to-word projection
c2w_en2pt = dill.load(open("results/c2w_en2pt_feproject.dill", 'rb'))

# word-to-word projection
w2w_en2pt = dill.load(open("results/w2w_en2pt_feproject.dill", 'rb'))

In [ ]:
print("Word-to-Word Projection:")
eval_res(gold_en2pt, w2w_en2pt)
print("Constituent-to-Word Projection:")
eval_res(gold_en2pt, c2w_en2pt)
print("Constituent-to-Constituent projection:")
eval_res(gold_en2pt, c2c_en2pt)

Word-to-Word Projection:
Total Hamming Loss: 0.187645
Hard Hamming Loss: 0.166759
Soft Hamming Loss: 0.020886
Exact Match Ratio: 0.121649
Constituent-to-Word Projection:
Total Hamming Loss: 0.208200
Hard Hamming Loss: 0.203337
Soft Hamming Loss: 0.004862
Exact Match Ratio: 0.167010
Constituent-to-Constituent projection:
Total Hamming Loss: 0.204000
Hard Hamming Loss: 0.179688
Soft Hamming Loss: 0.024312
Exact Match Ratio: 0.164948


In [ ]:
compare_projection(gold_en2pt, c2w_en2pt, c2c_en2pt)

752 Social_event ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', '-', '-', '-', '-', 'Social_event', 'Social_event', '-', '-', '-', '-']
758 Calendric_unit ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['Salient_event', 'Salient_event', '-', '-', 'Salient_event', 'Salient_event', 'Salient_event', 'Salient_event', '-', 'Salient_event', 'Salient_event', '-', '-', '-', '-', 'Unit', 'Unit', 'Unit', 'Unit', '-', '-']
763 Cogitation ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', 'Topic', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
764 Political_locales ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Locale', '-', '-', '-', '-', '-', '-']
773 People

### Analysis of Frame Elements Baseline

In [49]:
from globalfn.annotations import all_annotations, annotation
from globalfn.alignments import all_alignments, aligned_with
from nltk.corpus import framenet as fn

In [50]:
# map each frame to its set of core FEs
frame_to_coreFEs = collections.defaultdict(set)
for frame in fn.frames():
    for FE in frame.FE:
        if frame.FE[FE].coreType == 'Core':
            frame_to_coreFEs[frame.name].add(str(FE))
print(frame_to_coreFEs)

defaultdict(<class 'set'>, {'Abandonment': {'Theme', 'Agent'}, 'Abounding_with': {'Location', 'Theme'}, 'Absorb_heat': {'Container', 'Heat_source', 'Entity'}, 'Abundance': {'Place', 'Quantity', 'Collection'}, 'Abusing': {'Victim', 'Abuser'}, 'Access_scenario': {'Useful_location', 'Barrier', 'Theme'}, 'Accompaniment': {'Participants', 'Co-participant', 'Participant'}, 'Accomplishment': {'Goal', 'Agent'}, 'Accoutrements': {'Accoutrement'}, 'Accuracy': {'Deviation', 'Means', 'Agent', 'Degree', 'Instrument'}, 'Achieving_first': {'Cognizer', 'New_idea'}, 'Active_substance': {'Effect', 'Substance'}, 'Activity': {'Agent'}, 'Activity_abandoned_state': {'Activity', 'Agent'}, 'Activity_done_state': {'Activity', 'Agent'}, 'Activity_finish': {'Activity', 'Agent'}, 'Activity_ongoing': {'Activity', 'Duration', 'Agent'}, 'Activity_pause': {'Activity', 'Agent'}, 'Activity_paused_state': {'Activity', 'Agent'}, 'Activity_prepare': {'Activity', 'Agent'}, 'Activity_ready_state': {'Salient_entity', 'Protag

In [53]:
# EN-PT
def analyze_fes_baseline(src_lang, tgt_lang):
    count_allFE_match = 0
    count_coreFE_match = 0
    count_total = 0
    sentence_pair = 0
    for src_ids, tgt_ids in all_alignments(f"{src_lang}-{tgt_lang}").items():
        if type(src_ids) is int and len(tgt_ids) == 1:
            if annotation(src_ids) and annotation(tgt_ids[0]):
                for src_anno in annotation(src_ids):
                    for tgt_anno in annotation(tgt_ids[0]):
                        if src_anno.frameName == tgt_anno.frameName:
                            count_total += 1
                            src_core_FEs = [FE for FE in src_anno.tokenized_fe_idx if FE in frame_to_coreFEs[src_anno.frameName]]
                            tgt_core_FEs = [FE for FE in tgt_anno.tokenized_fe_idx if FE in frame_to_coreFEs[tgt_anno.frameName]]
                            if set(src_anno.tokenized_fe_idx) == set(tgt_anno.tokenized_fe_idx):
                                count_allFE_match += 1
                            
                            if set(src_core_FEs) == set(tgt_core_FEs):
                                count_coreFE_match += 1
    print("Percentage of Matching for All FEs:", count_allFE_match/count_total)
    print("Percentage of Matching for Core FEs:", count_coreFE_match/count_total)

In [54]:
analyze_fes_baseline("en", "pt")

1008 (language en) is not annotated.
1009 (language en) is not annotated.
1020 (language en) is not annotated.
1048 (language en) is not annotated.
1049 (language en) is not annotated.
1097 (language en) is not annotated.
1103 (language en) is not annotated.
1105 (language en) is not annotated.
861 (language pt) is not annotated.
1138 (language en) is not annotated.
1151 (language en) is not annotated.
1153 (language en) is not annotated.
1161 (language en) is not annotated.
1221 (language en) is not annotated.
1232 (language en) is not annotated.
Percentage of Matching for All FEs: 0.3278955954323002
Percentage of Matching for Core FEs: 0.4078303425774878


In [55]:
analyze_fes_baseline("en", 'de')

1008 (language en) is not annotated.
1009 (language en) is not annotated.
1020 (language en) is not annotated.
1048 (language en) is not annotated.
1049 (language en) is not annotated.
1067 (language en) is not annotated.
1340 (language de) is not annotated.
1341 (language de) is not annotated.
1342 (language de) is not annotated.
1343 (language de) is not annotated.
1344 (language de) is not annotated.
1345 (language de) is not annotated.
1346 (language de) is not annotated.
1347 (language de) is not annotated.
1348 (language de) is not annotated.
1349 (language de) is not annotated.
1350 (language de) is not annotated.
1351 (language de) is not annotated.
1352 (language de) is not annotated.
1353 (language de) is not annotated.
1354 (language de) is not annotated.
1354 (language de) is not annotated.
1355 (language de) is not annotated.
1356 (language de) is not annotated.
1097 (language en) is not annotated.
1358 (language de) is not annotated.
1359 (language de) is not annotated.
1

### Create CoNLL Files

In [21]:
from globalfn.annotations import all_annotations, annotation
from globalfn.alignments import all_alignments, aligned_with
from nltk.corpus import framenet as fn

In [7]:
"""
Load Universal and Fine-grained Parts-of-Speech taggers
"""
from flair.models import SequenceTagger
from flair.data import Sentence
upos_tagger = SequenceTagger.load('upos-fast')
# pos_tagger = SequenceTagger.load('pos-fast')

OSError: HEAD request failed for url https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/release-pos-fast-0/en-pos-ontonotes-fast-v0.4.pt with status code 301.

In [12]:
# map each frame to its set of core FEs
frame_to_coreFEs = collections.defaultdict(set)
for frame in fn.frames():
    for FE in frame.FE:
        if frame.FE[FE].coreType == 'Core':
            frame_to_coreFEs[frame.name].add(str(FE))
print(frame_to_coreFEs)

defaultdict(<class 'set'>, {'Abandonment': {'Theme', 'Agent'}, 'Abounding_with': {'Location', 'Theme'}, 'Absorb_heat': {'Container', 'Heat_source', 'Entity'}, 'Abundance': {'Place', 'Quantity', 'Collection'}, 'Abusing': {'Victim', 'Abuser'}, 'Access_scenario': {'Useful_location', 'Barrier', 'Theme'}, 'Accompaniment': {'Participants', 'Co-participant', 'Participant'}, 'Accomplishment': {'Goal', 'Agent'}, 'Accoutrements': {'Accoutrement'}, 'Accuracy': {'Deviation', 'Means', 'Agent', 'Degree', 'Instrument'}, 'Achieving_first': {'Cognizer', 'New_idea'}, 'Active_substance': {'Effect', 'Substance'}, 'Activity': {'Agent'}, 'Activity_abandoned_state': {'Activity', 'Agent'}, 'Activity_done_state': {'Activity', 'Agent'}, 'Activity_finish': {'Activity', 'Agent'}, 'Activity_ongoing': {'Activity', 'Duration', 'Agent'}, 'Activity_pause': {'Activity', 'Agent'}, 'Activity_paused_state': {'Activity', 'Agent'}, 'Activity_prepare': {'Activity', 'Agent'}, 'Activity_ready_state': {'Salient_entity', 'Protag

In [41]:
def get_FE_id(frame_name, FE_name):
    """
    Get the ID for the input frame element given the semantic frame
    """
    if FE_name == "-":
        return "-"
    frame = fn.frames(frame_name)[0]
    if FE_name not in frame.FE:
        return 0
    fe = frame.FE[FE_name]
    return fe.ID

def get_frame_id(frame_name):
    """
    Get the ID for the input semantic frame
    """
    if frame_name == "-":
        return "-"
    frame = fn.frames(frame_name)[0]
    return frame.ID

In [43]:
def conll_FE(src_lang, tgt_lang, frame_to_coreFEs):
    """
    Create files between src_lang and tgt_lang in the pseudo-CoNLL format 
    for further research on FE Divergences, where an aligned sentence pair 
    shares the same semantic frame but labeled with different frame elements.
    """
    count_allFE_match = 0  # count number of aligned sentences with all matching FEs
    count_coreFE_match = 0  # count number of aligned sentences with all matching core FEs
    count_total = 0  # total number of aligned sentences
    for src_ids, tgt_ids in all_alignments(f"{src_lang}-{tgt_lang}").items():
        if type(src_ids) is int and len(tgt_ids) == 1:
            if annotation(src_ids) and annotation(tgt_ids[0]):
                for src_anno in annotation(src_ids):
                    for tgt_anno in annotation(tgt_ids[0]):
                        if src_anno.frameName == tgt_anno.frameName:
                            # source sentences
                            src_tokenized_text = src_anno.tokenized_text.split(' ')
                            src_sent = Sentence(src_anno.tokenized_text, use_tokenizer=False)
                            pos_tagger.predict(src_sent)
                            src_pos_tokenized_text = [tok.get_labels()[0].value for tok in src_sent]
                            # upos_tagger.predict(src_sent)
                            src_upos_tokenized_text = [tok.get_labels()[0].value for tok in src_sent]
                            src_is_core_FE = [FE in frame_to_coreFEs[src_anno.frameName] for FE in src_anno.tokenized_fe_idx]
                            src_frame_ids = [get_frame_id(frame_name) for frame_name in src_anno.tokenized_frame_idx]
                            src_fe_ids = [get_FE_id(src_anno.frameName, FE) for FE in src_anno.tokenized_fe_idx]

                            # target sentences
                            tgt_tokenized_text = tgt_anno.tokenized_text.split(' ')
                            tgt_sent = Sentence(tgt_anno.tokenized_text, use_tokenizer=False)
                            pos_tagger.predict(tgt_sent)
                            tgt_pos_tokenized_text = [tok.get_labels()[0].value for tok in tgt_sent]
                            # upos_tagger.predict(tgt_sent)
                            tgt_upos_tokenized_text = [tok.get_labels()[0].value for tok in tgt_sent]
                            tgt_is_core_FE = [FE in frame_to_coreFEs[tgt_anno.frameName] for FE in tgt_anno.tokenized_fe_idx]
                            tgt_frame_ids = [get_frame_id(frame_name) for frame_name in tgt_anno.tokenized_frame_idx]
                            tgt_fe_ids = [get_FE_id(tgt_anno.frameName, FE) for FE in tgt_anno.tokenized_fe_idx]
                            
                            # create the CoNLL-format for the source and target sentences
                            with open(f"results_fe_baseline_projections/{src_lang}-{tgt_lang}/{src_lang}_{tgt_lang}_{src_ids}_{tgt_ids[0]}_{src_anno.annoID}_{tgt_anno.annoID}.tsv", 'w+') as wf:
                                for line in zip(src_tokenized_text, src_upos_tokenized_text, src_pos_tokenized_text, src_anno.tokenized_lu_idx, src_anno.tokenized_frame_idx, src_frame_ids, src_anno.tokenized_fe_idx, src_fe_ids, src_is_core_FE):
                                    line = map(str, line)
                                    line = '\t'.join(line) + '\n'
                                    wf.write(line)
                                wf.write('\n')
                                for line in zip(tgt_tokenized_text, tgt_upos_tokenized_text, tgt_pos_tokenized_text, tgt_anno.tokenized_lu_idx, tgt_anno.tokenized_frame_idx, tgt_frame_ids, tgt_anno.tokenized_fe_idx, tgt_fe_ids, tgt_is_core_FE):
                                    line = map(str, line)
                                    line = '\t'.join(line) + '\n'
                                    wf.write(line)
                            
                            # note down the alignments that have all matching FEs and core FEs
                            src_core_FEs = [FE for FE in src_anno.tokenized_fe_idx if FE in frame_to_coreFEs[src_anno.frameName]]
                            tgt_core_FEs = [FE for FE in tgt_anno.tokenized_fe_idx if FE in frame_to_coreFEs[tgt_anno.frameName]]
                            if set(src_anno.tokenized_fe_idx) == set(tgt_anno.tokenized_fe_idx):
                                count_allFE_match += 1
                                with open(f"/home/jovyan/work/results_fe_baseline_projections/{src_lang}-{tgt_lang}/allFE_divergence.txt", "a+") as af:
                                    af.write(f"{src_lang}_{tgt_lang}_{src_ids}_{tgt_ids[0]}_{src_anno.annoID}_{tgt_anno.annoID}.tsv\n")
                            if set(src_core_FEs) == set(tgt_core_FEs):
                                count_coreFE_match += 1
                                with open(f"/home/jovyan/work/results_fe_baseline_projections/{src_lang}-{tgt_lang}/coreFE_divergence.txt", "a+") as af:
                                    af.write(f"{src_lang}_{tgt_lang}_{src_ids}_{tgt_ids[0]}_{src_anno.annoID}_{tgt_anno.annoID}.tsv\n")
                            else:
                                print("Frame:", src_anno.frameName)
                                print(src_anno.tokenized_text)
                                print(src_anno.tokenized_fe_idx)
                                print(tgt_anno.tokenized_text)
                                print(tgt_anno.tokenized_fe_idx)


In [44]:
conll_FE("en", "de", frame_to_coreFEs)

1008 (language en) is not annotated.
1009 (language en) is not annotated.
Frame: Ranked_expectation
I 've been blown away by the whole thing .
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Die ganze Sache hier hat mich umgehauen .
['-', '-', 'Entity', '-', '-', '-', '-', '-']
Frame: Topic
There have been three themes running through the conference which are relevant to what I want to talk about .
['-', '-', '-', '-', 'Topic', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Es gab drei Leitmotive , die sich durch die Konferenz zogen , die wichtig sind für das , worüber ich sprechen will .
['-', '-', '-', '-', '-', '-', '-', '-', 'Text', 'Text', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Frame: Discussion
There have been three themes running through the conference which are relevant to what I want to talk about .
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Topic', 'Interlocutor_1', '-', '-', '-', 'Topic', '-']
E

In [45]:
conll_FE("en", "pt", frame_to_coreFEs)

Truthfully , what happens is , as children grow up , we start to educate them progressively from the waist up .
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Agent', '-', 'Activity', 'Activity', 'Activity', 'Activity', 'Activity', 'Activity', 'Activity', 'Activity', '-']
Sério , o que acontece é que à medida que as crianças crescem , nós começamos a educá-las progressivamente da cintura para cima .
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Agent', '-', '-', '-', 'Manner', 'Explanation', 'Explanation', 'Explanation', 'Explanation', '-']
Frame: Education_teaching
Truthfully , what happens is , as children grow up , we start to educate them progressively from the waist up .
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'Teacher', '-', '-', '-', 'Student', 'Manner', 'Depictive', 'Depictive', 'Depictive', 'Depictive', '-']
Sério , o que acontece é que à medida que as crianças crescem , nós começamos a educá-las progressivamente da cintura p